In [1]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client
from IPython.display import clear_output

client = Client(api_key="d9yKX3gEa9Hn0jOiVKTte8B55VRmwmnG4QYbpXO59UEWDyKSKf1XcyoYX2SYBwT8",api_secret="PLCQtT9PeDmZMenRmdUj4ONoPDv5Ph1xHdO8IFOCSK2BXfNXXts9JfFU56MKL8fz",tld='com') # Cristian Real

###################################
# Parámetros de estrategia.
###################################
smooth_interval = 20
smooth_exp = 3
std_mult = 0.25
pdi_mult = 15
mapd_mult = 25
sdi_mult = 50
k = 0

###################################
# Metadatos de la estrategia.
###################################
estrategia = 'DERIVATIVES_1SECOND_SPOT_V1'
trade_qty = 0.0001
bullish = False
bearish = False
df_logs = pd.DataFrame(columns=['trade_id','orden_id','client_order_id','cantidad','cantidad_ejecutada','accion','tiempo_op','precio_op','tiempo_sl','precio_sl','comision','activo_comision'])
fecha = '2024-04-17 18:00:00'
last_time_buy = datetime.utcnow()
last_time_sell = datetime.utcnow()
PNL = 0


while True:
    # clear_output(wait=True)
    ##################################################################################################################################
    # Descarga y limpieza de datos de X días atrás a hoy.
    ##################################################################################################################################
    recent_data = pd.read_csv('../../../../csvs/streaming/BTCUSDT.csv')
    recent_data.columns = ['Tiempo','Simbolo','Open']
    recent_data = recent_data[-60*5:]
    recent_data = recent_data.loc[:,['Tiempo','Open']]
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Open'] = recent_data.Open.astype(float)
    recent_data = recent_data.reset_index()
    
    if k == 0:
        time = datetime.utcnow()
        tiempo_inicio = f'{str(time)[:10]}-{str(time)[11:13]}-{str(time)[14:16]}-{str(time)[17:19]}'
        print(f'Inicia el bot de trading con inicio en: {time}')
        k += 1

    ##################################################
    # Interpolación (suavizado)  del Precio (OPEN) con SciPy y obtención de derivadas con su respectiva interpolación.
    ##################################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Open,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['MAPD'] = recent_data.PD_Interpolacion.rolling(window=15).mean().shift(1)
    mp = recent_data.Open.mean()


    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    # initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[:1].item()),'%Y-%m-%d %H:%M:%S')].index.item() + rango
    # final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S')].index.item() - rango

    abs_open_std = abs(np.std(recent_data.Open)*std_mult)
    pdi_cond = recent_data.PD_Interpolacion*pdi_mult
    sdi_cond = recent_data.SD_Interpolacion
    
    recent_data['sdi_updown'] = np.select(condlist=[(abs(sdi_cond*sdi_mult) > abs_open_std)],
                                                        choicelist=[1],
                                                        default=0)
    recent_data['pdi_updown'] = np.select(condlist=[(pdi_cond > abs_open_std),
                                                        (pdi_cond < -abs_open_std)],
                                                        choicelist=[1,-1],
                                                        default=0)
    recent_data['sell_buy_side'] = np.select(condlist=[sdi_cond > 0,
                                                        sdi_cond < 0],
                                                        choicelist=[1,-1],
                                                        default=0)
    
    recent_data['signal'] = 0
    recent_data.loc[(recent_data.sell_buy_side == 1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == 1) & (mp + recent_data.MAPD*mapd_mult > mp + abs_open_std*0.5),'signal'] = 1
    recent_data.loc[(recent_data.sell_buy_side == -1) & (recent_data.sdi_updown == 1) & (recent_data.pdi_updown == -1) & (mp + recent_data.MAPD*mapd_mult < mp),'signal'] = -1

#############################################
    # AUTOMATIC TRADING.
    #############################################
    try:
        last_buy_signal = recent_data.Tiempo[recent_data.signal == 1].iloc[-1:].item()
        last_buy_price = recent_data.Open[recent_data.signal == 1].iloc[-1:].item()
        last_sell_signal = recent_data.Tiempo[recent_data.signal == -1].iloc[-1:].item()
        last_sell_price = recent_data.Open[recent_data.signal == -1].iloc[-1:].item()
    except:
        pass
    
    if bullish == False:
        if (last_buy_signal != last_time_buy) & (last_buy_signal > last_time_buy) & (last_buy_signal > last_time_sell):
            last_time_buy = last_buy_signal
            last_price_buy = last_buy_price
            
            marketorder_buy = client.create_order(
                symbol='BTCFDUSD',
                side=Client.SIDE_BUY,
                type=Client.ORDER_TYPE_MARKET,
                quantity=trade_qty,
                recvWindow=60000)
            
            entry_price = float(marketorder_buy['fills'][0]['price'])
            entry_time = datetime.fromtimestamp(marketorder_buy['transactTime']/1000)
            TID = marketorder_buy['fills'][0]['tradeId']
            OID = marketorder_buy['orderId'] # orderId
            COID = marketorder_buy['clientOrderId'] # clientOrderId
            Qty = marketorder_buy['origQty'] # origQty
            exc_Qty = marketorder_buy['executedQty']
            side = marketorder_buy['side'] # Side
            commission = marketorder_buy['fills'][0]['commission'] # Commission
            commissionAsset = marketorder_buy['fills'][0]['commissionAsset'] # commissionAsset
            tiempo_final = f'{str(entry_time)[:10]}-{str(entry_time)[11:13]}-{str(entry_time)[14:16]}-{str(entry_time)[17:19]}'

            lista_logs = [TID,OID,COID,Qty,exc_Qty,side,entry_time,entry_price,last_time_buy,last_price_buy,commission,commissionAsset]
            df_logs.loc[len(df_logs)] = lista_logs
            
            bullish = True
            # bearish = False
            print(f'TIEMPO SEÑAL COMPRA: Precio; {last_price_buy}. Tiempo; {last_time_buy}. TIEMPO COMPRA: Precio; {entry_price}. Tiempo; {entry_time}.')
            print(f'DIFERENCIA ENTRE SEÑAL Y COMPRA: {(entry_time - last_time_buy+timedelta(hours=6)).seconds} segundos.') 
    
    elif bullish == True:
        if (last_sell_signal != last_time_sell) & (last_sell_signal > last_time_sell) & (last_sell_signal > last_time_buy):
            last_time_sell = last_sell_signal
            last_price_sell = last_sell_price

            marketorder_sell = client.create_order(
                symbol='BTCFDUSD',
                side=Client.SIDE_SELL,
                type=Client.ORDER_TYPE_MARKET,
                quantity=trade_qty,
                recvWindow=60000)
            
            exit_price = float(marketorder_sell['fills'][0]['price'])
            exit_time = datetime.fromtimestamp(marketorder_sell['transactTime']/1000)
            TID = marketorder_sell['fills'][0]['tradeId']
            OID = marketorder_sell['orderId'] # orderId
            COID = marketorder_sell['clientOrderId'] # clientOrderId
            Qty = marketorder_sell['origQty'] # origQty
            exc_Qty = marketorder_sell['executedQty']
            side = marketorder_sell['side'] # Side
            commission = marketorder_sell['fills'][0]['commission'] # Commission
            commissionAsset = marketorder_sell['fills'][0]['commissionAsset'] # commissionAsset
            profit = (exit_price-entry_price)*trade_qty
            PNL += profit
            tiempo_final = f'{str(exit_time)[:10]}-{str(exit_time)[11:13]}-{str(exit_time)[14:16]}-{str(exit_time)[17:19]}'
            
            lista_logs = [TID,OID,COID,Qty,exc_Qty,side,exit_time,exit_price,last_time_sell,last_price_sell,commission,commissionAsset]
            df_logs.loc[len(df_logs)] = lista_logs
            
            bullish = False
            # bearish = True
            print(f'TIEMPO SEÑAL VENTA: Precio; {last_price_sell}. Tiempo; {last_time_sell}. TIEMPO VENTA: Precio: {exit_price}. Tiempo: {exit_time}. tradePNL: {profit}, totalPNL: {PNL} FDUSD')
            print(f'DIFERENCIA ENTRE SEÑAL Y VENTA: {(exit_time - last_time_sell+timedelta(hours=6)).seconds} segundos.')

    if recent_data.Tiempo.iloc[-1:].item() >= datetime.strptime(fecha,'%Y-%m-%d %H:%M:%S'):
        df_logs.to_csv(
            path_or_buf = f'../LOGS/{estrategia}_{tiempo_inicio}_{tiempo_final}.csv',
            mode='w',
            index=False,
            header=True
            )
        break
# except:
#     df_logs.to_csv(
#         path_or_buf = f'../LOGS/{estrategia}_{tiempo_inicio}_{tiempo_final}.csv',
#         mode='w',
#         index=False,
#         header=True
#         )
#     if recent_data.Tiempo.iloc[-1:].item() >= date
# time.strptime('2024-04-13 21:00:00','%Y-%m-%d %H:%M:%S'):
#         break

Inicia el bot de trading con inicio en: 2024-04-17 05:28:04.789110
TIEMPO SEÑAL COMPRA: Precio; 64199.17. Tiempo; 2024-04-17 05:28:12.839000. TIEMPO COMPRA: Precio; 64253.33. Tiempo; 2024-04-16 23:28:13.130000.
DIFERENCIA ENTRE SEÑAL Y COMPRA: 0 segundos.
TIEMPO SEÑAL VENTA: Precio; 64183.2. Tiempo; 2024-04-17 05:28:25.225000. TIEMPO VENTA: Precio: 64225.31. Tiempo: 2024-04-16 23:28:25.552000. tradePNL: -0.0028020000000004075, totalPNL: -0.0028020000000004075 FDUSD
DIFERENCIA ENTRE SEÑAL Y VENTA: 0 segundos.
TIEMPO SEÑAL COMPRA: Precio; 64183.19. Tiempo; 2024-04-17 05:29:20.494000. TIEMPO COMPRA: Precio; 64237.99. Tiempo; 2024-04-16 23:29:20.824000.
DIFERENCIA ENTRE SEÑAL Y COMPRA: 0 segundos.
TIEMPO SEÑAL VENTA: Precio; 64197.47. Tiempo; 2024-04-17 05:29:51.062000. TIEMPO VENTA: Precio: 64236.96. Tiempo: 2024-04-16 23:29:51.708000. tradePNL: -0.0001029999999998836, totalPNL: -0.002905000000000291 FDUSD
DIFERENCIA ENTRE SEÑAL Y VENTA: 0 segundos.
TIEMPO SEÑAL COMPRA: Precio; 64180.0. T

In [24]:
df_logs.to_csv(
            path_or_buf = f'../LOGS/{estrategia}_{tiempo_inicio}_{tiempo_final}.csv',
            mode='w',
            index=False,
            header=True
            )

In [18]:
recent_data.Tiempo[recent_data.signal == -1].iloc[-1:]

Series([], Name: Tiempo, dtype: datetime64[ns])